In [2]:
import chromedriver_autoinstaller

In [3]:
# pip install selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

In [50]:
youtube = []
driver = webdriver.Chrome()
url = "https://www.youtube.com/results?search_query=artificial+intelligence"
driver.get(url)
options = webdriver.ChromeOptions()
options.add_argument("--incognito")

wait = WebDriverWait(driver, 10)
cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
cookie_consent.click()

#while True:
#    scroll_height = 10000
#    document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
#    driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
#    time.sleep(3)
#    document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
#    if document_height_after == document_height_before:
#        break

# Extract the HTML and parse it with BeautifulSoup
html = driver.execute_script("return document.body.outerHTML;")
soup = BeautifulSoup(html, "html.parser")

video_name = soup.find("yt-formatted-string", class_="style-scope ytd-video-renderer")
video_title = video_name.getText().strip()
video_link = [a.get('href') for a in soup.find_all('a', class_= 'yt-simple-endpoint style-scope ytd-video-renderer', href=True)]
#video_views = soup.find_all("span", class_="inline-metadata-item style-scope ytd-video-meta-bloc")
#video_upload = soup.find_all("span", class_="inline-metadata-item style-scope ytd-video-meta-bloc")



# Zip to add all values in a tuple for each restaurant
rows = list(zip(video_title))
#, video_link, video_views, video_upload))
youtube.extend(rows)
        
df_yt = pd.DataFrame(youtube, columns=['Name'])



In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re


In [25]:

driver = webdriver.Chrome()
url = "https://www.youtube.com/results?search_query=artificial+intelligence"
driver.get(url)
options = webdriver.ChromeOptions()
options.add_argument("--incognito")

wait = WebDriverWait(driver, 10)
cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
cookie_consent.click()

html = driver.execute_script("return document.body.outerHTML;")
soup = BeautifulSoup(html, "html.parser")

videos = soup.find_all("div", class_="text-wrapper style-scope ytd-video-renderer")

rows = []
for video in videos:
    name = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").text
    link = "https://www.youtube.com" + video.find("a", class_="yt-simple-endpoint style-scope ytd-video-renderer")['href']
    info = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").get("aria-label").split(" by ")
    views = re.search(r'([\d,]+)\s*(views|view)', info[1]).group(1)

    try:
        date = re.search(r'(?P<count>\d+)\s+(?P<unit>\w+)\s+ago', info[1]).group()
    except:
        date = 'Unknown'
    
    try:
        duration =  re.search(r'\d+\s+(?:(?:minute|min|second|sec)s?(?:,\s*)?)+\d*\s*(?:seconds|second?)?', info[1]).group()
    except:
        duration = 'Unknown'



    rows.append([name, views, duration, date, link])
        
df_yt = pd.DataFrame(rows, columns=['Name', 'views', 'duration', 'date', 'link'])


'TheAIGRID 8 days ago 16 minutes 2,604 views'

In [26]:
df_yt

,Name,views,duration,date,link
0,What Is Artificial Intelligence? | 🔥Artificial...,"1,158,830","5 minutes, 28 seconds",3 years ago,https://www.youtube.com/watch?v=ad79nYk2keg
1,How will AI change the world?,"704,808","5 minutes, 56 seconds",3 months ago,https://www.youtube.com/watch?v=RzkD_rTEBYs
2,Steven Spielberg Talks Artificial Intelligence...,"33,617","3 minutes, 37 seconds",16 hours ago,https://www.youtube.com/watch?v=tSoCtw5vNS0
3,How Artificial Intelligence is Changing Our Wo...,"1,175",44 minutes,10 hours ago,https://www.youtube.com/watch?v=-PztBI2lG54
4,Artificial Intelligence: Last Week Tonight wit...,"5,039,396",27 minutes,9 days ago,https://www.youtube.com/watch?v=Sqa8Zo2XWc4
5,"ChatGPT: Artificial Intelligence, chatbots and...","125,169","13 minutes, 22 seconds",1 day ago,https://www.youtube.com/watch?v=1wzPr4cUoMQ
6,"Generative AI Is About To Reset Everything, An...","513,882",21 minutes,2 weeks ago,https://www.youtube.com/watch?v=WY518YRfs5M
7,Artificial intelligence and algorithms: pros a...,"10,005,730",42 minutes,3 years ago,https://www.youtube.com/watch?v=s0dMTAQM4cw
8,Artificial Intelligence.,"146,741",30 minutes,4 days ago,https://www.youtube.com/watch?v=_iMItrc0ChU
9,The Danger of AI | Scary Technology | Artifici...,"63,030",52 minutes,13 days ago,https://www.youtube.com/watch?v=5guiUrgcblM


In [48]:
from selenium.common.exceptions import NoSuchElementException
import time

In [67]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

driver = webdriver.Chrome()
url = "https://www.youtube.com/watch?v=ad79nYk2keg"
driver.get(url)
options = webdriver.ChromeOptions()
options.add_argument("--incognito")

wait = WebDriverWait(driver, 10)
cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
cookie_consent.click()
time.sleep(30)
driver.execute_script("window.scrollBy(0, 500);")
time.sleep(5)

#while True:
#    scroll_height = 10
#    document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
#    driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
#    time.sleep(3)
#    document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
#    if document_height_after == document_height_before:
#        break


html = driver.execute_script("return document.body.outerHTML;")
soup = BeautifulSoup(html, "html.parser")

comments = soup.find_all("div", class_="style-scope ytd-comment-renderer")


rows = []
for comment in comments:
    yt_comment = comment.find("yt-formatted-string", class_="style-scope ytd-comment-renderer")

    rows.append([yt_comment])
        
df_yt_comment = pd.DataFrame(rows, columns=['Comment'])


In [68]:
df_yt_comment

,Comment
0,None
1,None
2,"[[], [Explore Our FREE Courses With Completion..."
3,None
4,"[[], [Explore Our FREE Courses With Completion..."
...,...
116,None
117,None
118,[Nice video. A quick learn about AI. Thank you...
119,None
